In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 530 Mini Project 1 
**Data Source**
The Metropolitan Museum of Art Open Access, The Metropolitan Museum of Art @ Kaggle. https://www.kaggle.com/metmuseum/the-metropolitan-museum-of-art-open-access

## 1. Overview
> containing in which you describe your research area and questions
1. User input a keyword, see how artworks that contain that keyword are distributed across time.
2. The frequencies of different mediums being used for artworks that contain the keyword.
3. The frequencies of a medium being used for artworks that contain the keyword and their distribution across time.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

## 2. Data Profile
**Data I need:** Object ID, Is Highlight, Department, Object Name, Title, Artist Display Name, Artist Nationality, Object Date, Object Begin Date, Object End Date, Medium

In [ ]:
met = pd.read_csv('../input/the-metropolitan-museum-of-art-open-access/MetObjects.csv')
met.info()

In [ ]:
# Only keep columns I need
met_df = met[['Object ID', 'Is Highlight', 'Department', 'Object Name', 'Medium', 'Title', 'Artist Display Name', 'Artist Nationality', 'Object Date', 'Object Begin Date', 'Object End Date']]
print(met_df.head())

## 3. Analysis
> consisting of your python code and Text Cells that explain how you have completed your project
1. Search through #Title and #ObjectName to locate artworks related to the keyword.
2. Do a scattered plot on a particular #Medium across the #ObjectTime.
3. Play with #IsHighlight, #Department, and #ArtistNationality to create parallel sets.

### Question1: How many artworks are related to "____"?
### Step1: Input a Keyword

In [ ]:
# Input a keyword, try "star"
#keyword = input("Type a keyword: ")
keyword = "star"

#Create a new dataframe collecting objects where either its "Title" or "Object Name"contains the keyword
met_keyword_df = met_df[
    (met_df['Title'].str.contains(f'\\b{keyword}\\b', case=False, regex=True)) | 
    (met_df['Object Name'].str.contains(f'\\b{keyword}\\b', case=False, regex=True))
]

met_keyword_df.info()
met_keyword_df.head()

In [ ]:
artwork = [keyword, 'all artwork']
number = [len(met_keyword_df), len(met_df)]
explode = (0.2, 0)

fig1, ax1 = plt.subplots()
ax1.pie(number, explode=explode, labels=artwork, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.show()

### Question2: For artworks related to "____", what is the distrubution of their object time and medium types?
### Step2: Map these objects containing the keyword and plot base on "Object Begin Date" and "Medium"

In [ ]:
fig2 = px.scatter(met_keyword_df, x = 'Object Begin Date', y='Medium')
fig2.show()

### Question3: For artworks related to "____", what is the distrubution and overlaps of their department and artist nationality?
### Step3: Map these objects containing the keyword and plot base on #IsHighlight, #Department, and #ArtistNationality to create parallel sets.

In [ ]:
fig3 = px.parallel_categories(met_keyword_df)
fig3.show()

## 4. Conclusions/Directions for future work
> consisting of any final thoughts and conclusions from your work, and any external references that are relevant to it

**Conclusions and Findings:** 
1. For the Fig2, it's interesting to see most of the "star" artworks made around 1000-1500 used stone and wool related mediums, and those around 1800-2000 used lithograph and engraving mediums. 
2. For the Fig3, it's clear that most of the "star" artworks are drawings and paintings, and the second biggest category of the "star" artworks is Islamic art.

**Future Work:** I would like to enable more keyword searches at the same time so that I can compare the portion of different {keyword} artworks and their distribution on other attributes.

# 530 Mini Project 2

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

mm = met[met['Medium'].notnull() & met['Artist Nationality'].notnull()]
mm = mm[(mm['Object Begin Date'] > -5000) & (mm['Object Begin Date'] <= 2021)]
x_m, x_a,y = mm['Medium'], mm['Artist Nationality'], mm['Object Begin Date']

# p = mm.groupby('Object Begin Date').count()['Object ID']
# print(p.index, p.values)
# plt.plot(p.index, p.values, label="by year")
# plt.show()

le_m = preprocessing.LabelEncoder()
x_labels = le_m.fit_transform(x_m)

le_a = preprocessing.LabelEncoder()
x_nationality = le_a.fit_transform(x_a)


x_input = list(zip(x_labels, x_nationality))

model = DecisionTreeClassifier()
model.fit(list(x_input), y)
y_pred = model.predict(x_input)

### Seems like it's not able to plot... is it because it's too big?

In [ ]:
# tree.plot_tree(model)

In [ ]:
diff = np.array(y.values) - np.array(y_pred)
print(diff.mean(), diff.max(), diff.min(), diff.std())